# Import Library

In [1]:
import requests
import json
from bs4 import BeautifulSoup

import pandas as pd
pd.set_option('max_rows',20)
import numpy as np
import xmltodict
import re

import datetime
from datetime import date, timedelta

from pmdarima import auto_arima
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima_model import ARIMA

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

import folium
from folium.plugins import MarkerCluster

import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

import warnings
warnings.filterwarnings('ignore')

# Scraping Weather Data at BMKG Web

In [2]:
# request weather data in bmkg website

url_cuaca = 'https://www.bmkg.go.id/cuaca/prakiraan-cuaca-indonesia.bmkg'
page = requests.get(url_cuaca)
content = BeautifulSoup(page.content, 'html.parser')

In [3]:
# scraping today's weather data in bmkg website

def get_weather_today(content):
    tab1 = content.find("div", id="TabPaneCuaca1")
    table = tab1.find(class_="table-prakicu-provinsi")
    data = table.find_all("tr")
    data = [data[0]]+data[3:]

    cities = [city.find("a").get_text() for city in data]
    weather_list = [weather.find("span").get_text() for weather in data]
    img_list = [img.find("img")['src'] for img in data]
    temp_list = [temp.find("td").find_next_siblings()[-2].get_text() for temp in data]
    humidity_list = [humidity.find("td").find_next_siblings()[-1].get_text() for humidity in data]

    data = pd.DataFrame({
        'Kota' : cities,
        'Perkiraan Cuaca' : weather_list,
        'Gambar' : img_list,
        'Cuaca (C)' : temp_list,
        'Kelembapan (%)' : humidity_list
    })
    return data

In [4]:
# scraping weather data tommorow in bmkg website

def get_weather_tomorrow(content):
    tab2 = content.find("div", id="TabPaneCuaca2")
    table = tab2.find(class_="table-prakicu-provinsi")
    data = table.find_all("tr")
    data = [data[0]]+data[3:]

    cities = [city.find("a").get_text() for city in data]
    weather_list = [weather.find("span").get_text() for weather in data]
    img_list = [img.find("img")['src'] for img in data]
    temp_list = [temp.find("td").find_next_siblings()[-2].get_text() for temp in data]
    humidity_list = [humidity.find("td").find_next_siblings()[-1].get_text() for humidity in data]

    data = pd.DataFrame({
        'Kota' : cities,
        'Perkiraan Cuaca' : weather_list,
        'Gambar' : img_list,
        'Cuaca (C)' : temp_list,
        'Kelembapan (%)' : humidity_list
    })
    return data

In [5]:
# scraping weather data the day after tomorrow in bmkg website

def get_weather_aftrtomorrow(content):
    tab3 = content.find("div", id="TabPaneCuaca3")
    table = tab3.find(class_="table-prakicu-provinsi")
    data = table.find_all("tr")
    data = [data[0]]+data[3:]

    cities = [city.find("a").get_text() for city in data]
    weather_list = [weather.find("span").get_text() for weather in data]
    img_list = [img.find("img")['src'] for img in data]
    temp_list = [temp.find("td").find_next_siblings()[-2].get_text() for temp in data]
    humidity_list = [humidity.find("td").find_next_siblings()[-1].get_text() for humidity in data]

    data = pd.DataFrame({
        'Kota' : cities,
        'Perkiraan Cuaca' : weather_list,
        'Gambar' : img_list,
        'Cuaca (C)' : temp_list,
        'Kelembapan (%)' : humidity_list
    })
    return data

In [6]:
todays_weather = get_weather_today(content)
tomorrows_weather = get_weather_tomorrow(content)
aftrtomorrows_weather = get_weather_aftrtomorrow(content)

# Show Today's Weather
todays_weather.head()

,Kota,Perkiraan Cuaca,Gambar,Cuaca (C),Kelembapan (%)
0,Banda Aceh,Cerah Berawan,https://www.bmkg.go.id/asset/img/weather_icon/...,23 - 32,65 - 90
1,Denpasar,Hujan Ringan,https://www.bmkg.go.id/asset/img/weather_icon/...,24 - 33,65 - 90
2,Serang,Berawan,https://www.bmkg.go.id/asset/img/weather_icon/...,24 - 30,60 - 90
3,Bengkulu,Berawan,https://www.bmkg.go.id/asset/img/weather_icon/...,24 - 32,60 - 95
4,Yogyakarta,Berawan,https://www.bmkg.go.id/asset/img/weather_icon/...,23 - 31,70 - 95


In [7]:
# function to get weather data by city
def get_weather(df,city='Makassar'):
    return df[df['Kota']==city]['Perkiraan Cuaca']

# function to get temperature data by city
def get_temperatur(df,city='Makassar'):
    return df[df['Kota']==city]['Cuaca (C)']

In [8]:
# Show Today's Weather in Makassar City
get_weather(todays_weather)

27    Hujan Sedang
Name: Perkiraan Cuaca, dtype: object

# Request Earthquake Data

In [9]:
# request earthquake data

url_gempa = "https://data.bmkg.go.id/gempadirasakan.xml"
response = requests.get(url_gempa)
data = xmltodict.parse(response.content)
json_data = json.dumps(data)

df_gempa = pd.read_json(json_data)
df_gempa = df_gempa['Infogempa']["Gempa"]
new_df = pd.json_normalize(df_gempa, meta_prefix=False)

new_df['point.coordinates'] = new_df['point.coordinates'].apply(lambda x: re.split(r', ', x))

earthquake_data = new_df
earthquake_data.head()

,Tanggal,Posisi,Magnitude,Kedalaman,_symbol,Keterangan,Dirasakan,point.coordinates
0,23/02/2021-21:09:10 WIB,9.48 LS 115.54 BT,4.3,37 Km,../gambar/k1.gif,Pusat gempa berada di laut 83 km tenggara Kuta...,"III\tLombok Tengah, III\tLombok Barat, II-III\...","[-9.48, 115.54]"
1,23/02/2021-02:22:09 WIB,1.5 LU 121.97 BT,5.8,10 Km,../gambar/k1.gif,Pusat gempa berada di laut 90 km TimurLaut Buol,"III\tBone Bolango, IV\tBuol, III\tToli - Toli,","[1.5, 121.97]"
2,22/02/2021-11:45:01 WIB,8.84 LS 123.54 BT,5.1,57 Km,../gambar/k1.gif,Pusat gempa berada di laut 41 km tenggara Lembata,"II\tEnde,","[-8.84, 123.54]"
3,22/02/2021-04:10:43 WIB,4.34 LS 102.13 BT,5,10 Km,../gambar/k1.gif,Pusat gempa berada di laut 57 km Barat Daya Se...,"III\tBengkulu, II\tSeluma, II\tKepahiang,","[-4.34, 102.13]"
4,21/02/2021-21:56:32 WIB,7.64 LS 106.56 BT,4.6,28 Km,../gambar/k1.gif,Pusat gempa berada di laut 72 km Selatan Sukabumi,"III\tCigaru, III\tPangandaran, III\tGarut Sela...","[-7.64, 106.56]"


In [10]:
# visualization earthquake data in folium map

def fig_gempa(df):
    world_map = folium.Map(location=[-2.548926, 118.0148634],
                      zoom_start=5,
                      tiles='openstreetmap')

    marker_cluster = MarkerCluster().add_to(world_map)

    for i in range(len(df)):
        lat = float(df['point.coordinates'].iloc[i][0])
        long = float(df['point.coordinates'].iloc[i][1])
        popup_text = """Tanggal : {}<br> Magnitude : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Tanggal'],
                    df.iloc[i]['Magnitude'])
        folium.Marker(location = [lat, long], 
                    popup=popup_text,
                    icon=folium.Icon(color="red", icon="circle", prefix="fa")).add_to(world_map)

    world_map.save('map.html')
    return world_map

In [11]:
fig_gempa(earthquake_data)

# Weather Forecating in Makassar City

In [12]:
# Load rainfall data

data = pd.read_excel('laporan_iklim_harian.xlsx', skiprows=8, usecols=[0,1], nrows=61)
data['RR'].replace(8888, np.nan, inplace=True)
data.fillna(method='ffill', inplace=True)
series = data.set_index('Tanggal')

weather = data['RR'].values
dates = data['Tanggal'].values

In [13]:
fig = px.line(data, x='Tanggal', y='RR', title='Curah Hujan')

fig.update_xaxes(
    rangeslider_visible=True
)
fig.show()

In [14]:
# cek stasionary data

stat, p, lags, critical_values = kpss(data['RR'], 'ct')

print('Test Statistics : ', stat)
print('p-value : ', p)
print('Critical value : ', critical_values)

if p < 0.05 :
    print('Time Series is not stasionary')
else:
    print('Time Series is stasionary')

Test Statistics :  0.10754133339532145
p-value :  0.1
Critical value :  {'10%': 0.119, '5%': 0.146, '2.5%': 0.176, '1%': 0.216}
Time Series is stasionary


In [15]:
stepwise_fit = auto_arima(series['RR'], trace=True,
                         suppress_warnings=True)
stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=599.907, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=593.095, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=594.595, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=641.332, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=594.623, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=594.541, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=596.514, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=604.014, Time=0.01 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.337 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   61
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -293.547
Date:                Tue, 23 Feb 2021   AIC                            593.095
Time:                        22:42:00   BIC                            599.427
Sample:                             0   HQIC                           595.576
                                 - 61                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     20.3318      7.635      2.663      0.008       5.368      35.295
ar.L1          0.3855      0.140      2.755      0.006       0.111       0.660
sigma2       883.6396    218.179      4.050      0.000     456.017    1311.262
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):                 5.96
Prob(Q):                              0.81   Prob(JB):                         0.05
Heteroskedasticity (H):               2.22   Skew:                             0.76
Prob(H) (two-sided):                  0.08   Kurtosis:                         2.78
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [16]:
# function for creating ARIMA model in rainfall data
def ARIMA_model(series, order, days = 14):
    # Fitting and forecast the series
    model = ARIMA(series, order = order)
    model = model.fit(disp=0)
    forecast, err, ci = model.forecast(steps = days, alpha = 0.05)
    start_day = data['Tanggal'].iloc[-1]
    predictions_df = pd.DataFrame({'Forecast':forecast.round()}, index=pd.date_range(start = start_day, periods=days, freq='D'))
    return predictions_df, ci

# function to display data forecast in a line chart
def plot_results(series, df_forecast, ci, label):
    import plotly.graph_objects as go
    start = data['Tanggal'].iloc[0]
    series = pd.DataFrame({'Real data':series}, index=pd.date_range(start = start, periods=series.shape[0], freq='D'))

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=series.index, y=series['Real data'],
        line_color='rgb(0,100,80)',
        name='Curah Hujan Harian',
    ))
    fig.add_trace(go.Scatter(
        x=df_forecast.index, y=df_forecast['Forecast'],
        line_color='Red',
        name='Prediksi Curah Hujan',
    ))

    annotations = []

    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                            xanchor='left', yanchor='bottom',
                            text='Prediksi Curah Hujan di Kota Makassar',
                            font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                            showarrow=False))
    # Source
    annotations.append(dict(xref='paper', yref='paper', x=0.7, y=-0.1,
                                xanchor='right', yanchor='top',
                                text='Sumber Data: https://dataonline.bmkg.go.id',
                                font=dict(family='Arial',
                                            size=12,
                                            color='rgb(150,150,150)'),
                                showarrow=False))


    fig.update_traces(mode='lines')
    fig.update_layout(annotations=annotations)
    return fig

In [17]:
# Order for ARIMA model
order = {
    'weather': (1, 0, 0),
}

# Stats of today
weather_today,  dates_today = weather[-1], dates[-1]

# Forecasting with ARIMA models
weather_pred, weather_ci = ARIMA_model(weather, order['weather'])

# plot_results(weather, weather_pred, weather_ci, 'Weather')
plot_results(weather, weather_pred, weather_ci, 'Weather')

In [18]:
def forecast():
    return dcc.Graph(id='forecast', figure=plot_results(weather, weather_pred, weather_ci, 'Weather'))

# Generate Dashboard 

In [19]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Perkiraan Cuaca Indonesia Dashboard'

colors = {
    'background': '#2f4245',
    'bodyColor':'#2f4245',
    'text': '#ffffff'
}

def get_page_heading_style():
    return {'backgroundColor': colors['background']}

def get_page_heading_title():
    return html.H1(children='PRAKIRAAN CUACA',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualisasi Data Perkiraan Cuaca Indonesia',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    header = (main_header,subtitle_header)
    return header

def get_city_list():
    return todays_weather['Kota'].unique()

def create_dropdown_list(city_list):
    dropdown_list = []
    for city in sorted(city_list):
        tmp_dict = {'label':city,'value':city}
        dropdown_list.append(tmp_dict)
    return dropdown_list

def get_city_dropdown():
    return html.Div([
                        html.Label('Select City', style={"color": "#ffffff"}),
                        dcc.Dropdown(id='my-id1',
                            options=create_dropdown_list(get_city_list()),
                            value='Makassar'
                        ),
                    ])

def generate_card_content(card_header,card_value,overall_value):
    card_head_style = {'textAlign':'center','fontSize':'150%'}
    card_body_style = {'textAlign':'center','fontSize':'100%'}
    card_header = dbc.CardHeader(card_header,style=card_head_style)
    card_body = dbc.CardBody(
        [
            html.H5((card_value), className="card-title",style=card_body_style),
            html.P(
                overall_value,
                className="card-text",style={'textAlign':'center'}
            ),
        ]
    )
    card = [card_header,card_body]
    return card

def generate_cards(city='Makassar'):
    regional_weather_today = get_weather(todays_weather,city)
    regional_weather_tomorrow = get_weather(tomorrows_weather,city)
    regional_weather_aftrtomorrow = get_weather(aftrtomorrows_weather,city)
    regional_temp_today = get_temperatur(todays_weather,city)
    regional_temp_tomorrow = get_temperatur(tomorrows_weather,city)
    regional_temp_aftrtomorrow = get_temperatur(aftrtomorrows_weather,city)

    today = date.today()
    yesterday = today - timedelta(days = 1)
    tomorrow = today + timedelta(days = 1)

    cards = html.Div(
        [
            dbc.Row(
                [
                    dbc.Col(dbc.Card(generate_card_content(yesterday.strftime("%d/%m/%Y"),regional_weather_today,regional_temp_today), color="success", inverse=True),md=dict(size=2,offset=3)),
                    dbc.Col(dbc.Card(generate_card_content(today.strftime("%d/%m/%Y"),regional_weather_tomorrow,regional_temp_tomorrow), color="warning", inverse=True),md=dict(size=2)),
                    dbc.Col(dbc.Card(generate_card_content(tomorrow.strftime("%d/%m/%Y"),regional_weather_aftrtomorrow,regional_temp_aftrtomorrow),color="danger", inverse=True),md=dict(size=2)),
                ],
                className="mb-4",
            ),
        ],id='card1'
    )
    return cards

In [ ]:
def generate_layout():
    page_header = generate_page_header()
    layout = dbc.Container(
        [
            html.Hr(),
            page_header[0],
            page_header[1],
            html.Hr(),
            dbc.Row(
                [
                    dbc.Col(get_city_dropdown(),md=dict(size=6,offset=3))                    
                ]

            ),
            html.Hr(),
            generate_cards(),
            html.Hr(),
            dbc.Row(
                [                
                    
                    dbc.Col(html.Label('Gempabumi Terkini', style={"color": "#ffffff"}), md=dict(size=6,offset=3))
        
                ],
            ),
            dbc.Row(
                [                
                    
                    dbc.Col(html.Iframe(srcDoc=open('map.html', 'r').read(), style={"width": "inherit", "height":"300px"}), md=dict(size=6,offset=3))
        
                ],
                align="center",

            ),
            html.Hr(),
            dbc.Row(
                [                
                    
                    dbc.Col(forecast(),md=dict(size=6,offset=3))
        
                ],
                align="center",

            ),

        ],fluid=True,style={'backgroundColor': colors['bodyColor']}
    )
    return layout

app.layout = generate_layout()

@app.callback(
        Output(component_id='card1',component_property='children'),
        Input(component_id='my-id1',component_property='value')
)
def update_output_div(value):
    return generate_cards(value)

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2021 22:42:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 22:42:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 22:42:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 22:42:15] "POST /_dash-update-component HTTP/1.1" 200 -
